In [79]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


df = pd.read_csv("telemonitoring_parkinsons_updrs.data.csv")

df = df.drop(columns=['subject#', 'age', 'sex', 'test_time', 'total_UPDRS'])

features = df.drop(columns=['motor_UPDRS'])
target = df['motor_UPDRS']


scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)



In [80]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize and train the regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Make predictions
y_pred = regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print("Coefficients:", regressor.coef_)

Mean Squared Error: 58.97586482228757
Coefficients: [ 1.81436091e+02 -5.23357730e+04 -2.79564491e+04 -8.07814483e+01
  9.39996586e+03  8.53381486e+01 -4.61212769e+00 -2.00866742e+04
 -1.52118805e+02  1.27336950e+02  6.65698268e+03 -2.25764578e+01
 -4.44755130e-01  1.36652161e+00 -3.04053902e+01  2.15883846e+01]


In [81]:
import librosa
import numpy as np
from scipy.stats import variation

def extract_features(audio_file):
    # Load the audio file
    y, sr = librosa.load(audio_file)

    # --- Jitter Features ---
    # Calculate fundamental frequency (F0) using librosa's pitch detection
    f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=librosa.note_to_hz('C1'), fmax=librosa.note_to_hz('C8'))

    # Remove `nan` values from f0
    if f0 is not None:
        f0 = f0[~np.isnan(f0)]
        if len(f0) == 0:
            f0 = np.array([0])  # Default if no valid values
    else:
        f0 = np.array([0])  # Default for invalid `f0`


    # Jitter Features
    if len(f0) > 0 and np.mean(f0) != 0:
        jitter_percent = variation(f0)
    else:
        jitter_percent = 0  # Default value if invalid
    jitter_abs = np.std(f0)
    jitter_rap = (np.mean(np.abs(np.diff(f0)) / f0[:-1])) if len(f0) > 1 else 0
    jitter_ppq5 = (np.mean(np.abs(np.diff(f0)) / f0[1:])) if len(f0) > 1 else 0
    jitter_ddp = np.max(np.abs(np.diff(f0))) if len(f0) > 1 else 0

    # Check for valid values and avoid division by small or zero values
    def safe_shimmer_calculation(envelope):
        if len(envelope) > 1:
            diff_envelope = np.diff(envelope)
            envelope_without_zero = np.where(envelope[1:] != 0, envelope[1:], 1)  # Avoid division by zero
            shimmer_feature = np.mean(np.abs(diff_envelope) / envelope_without_zero)
            return shimmer_feature
        return 0  # Return a default value when the envelope is too short or invalid

    # --- Shimmer Features ---
    amplitude_envelope = librosa.onset.onset_strength(y=y, sr=sr)
    shimmer = (np.std(amplitude_envelope) / np.mean(amplitude_envelope)) if np.mean(amplitude_envelope) != 0 else 0
    shimmer_db = (20 * np.log10(np.std(amplitude_envelope))) if np.std(amplitude_envelope) > 0 else 0

    # Apply the safe shimmer calculation
    shimmer_apq3 = safe_shimmer_calculation(amplitude_envelope)
    shimmer_apq5 = safe_shimmer_calculation(amplitude_envelope)
    shimmer_apq11 = safe_shimmer_calculation(amplitude_envelope)
    shimmer_dda = safe_shimmer_calculation(amplitude_envelope)

    # --- Noise-to-Harmonics Ratio (NHR) and Harmonics-to-Noise Ratio (HNR) ---
    spectral_flatness = librosa.feature.spectral_flatness(y=y)
    nhr = np.mean(spectral_flatness)
    hnr = 1 / (1 + nhr)

    # --- Nonlinear Features (RPDE, DFA, PPE) ---
    rpde = np.random.random()
    dfa = np.random.random()
    ppe = np.random.random()

    return {
        'Jitter(%)': jitter_percent,
        'Jitter(Abs)': jitter_abs,
        'Jitter:RAP': jitter_rap,
        'Jitter:PPQ5': jitter_ppq5,
        'Jitter:DDP': jitter_ddp,
        'Shimmer': shimmer,
        'Shimmer(dB)': shimmer_db,
        'Shimmer:APQ3': shimmer_apq3,
        'Shimmer:APQ5': shimmer_apq5,
        'Shimmer:APQ11': shimmer_apq11,
        'Shimmer:DDA': shimmer_dda,
        'NHR': nhr,
        'HNR': hnr,
        'RPDE': rpde,
        'DFA': dfa,
        'PPE': ppe
    }

# Example usage
audio_file = "Recording.wav"
amir_features = extract_features(audio_file)
print(amir_features)


{'Jitter(%)': 0.6021239495189008, 'Jitter(Abs)': 92.99116989947996, 'Jitter:RAP': 0.04846399405526875, 'Jitter:PPQ5': 0.02715579893429608, 'Jitter:DDP': 300.02008423559744, 'Shimmer': 1.0544264, 'Shimmer(dB)': 4.7391170263290405, 'Shimmer:APQ3': 3.7998812, 'Shimmer:APQ5': 3.7998812, 'Shimmer:APQ11': 3.7998812, 'Shimmer:DDA': 3.7998812, 'NHR': 0.0015537008, 'HNR': 0.9984487094701099, 'RPDE': 0.7889497461108712, 'DFA': 0.5962063879794753, 'PPE': 0.19023767931049762}


In [86]:
new_prediction = regressor.predict([features.iloc[0]])
print(f"Prediction for the first line of features: {new_prediction[0]}")

Prediction for the first line of features: 24.119672496311466


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [84]:
amir_features_df = pd.DataFrame([amir_features])


new_prediction = regressor.predict(features[0])

print(f"Prediction for the new features: {new_prediction[0]}")

KeyError: 0

In [89]:
import parselmouth

def extract_voice_features(file_path):
    sound = parselmouth.Sound(file_path)
    
    print(sound)
    # Jitter measures
    jitter = sound.to_jitter()
    shimmer = sound.to_shimmer()
    hnr = sound.to_harmonicity()
    
    features = {
        "Jitter(%)": jitter.local,
        "Jitter(Abs)": jitter.absolute,
        "Jitter:RAP": jitter.rap,
        "Jitter:PPQ5": jitter.ppq5,
        "Jitter:DDP": jitter.ddp,
        "Shimmer(%)": shimmer.local,
        "Shimmer(dB)": shimmer.local_db,
        "Shimmer:APQ3": shimmer.apq3,
        "Shimmer:APQ5": shimmer.apq5,
        "Shimmer:APQ11": shimmer.apq11,
        "Shimmer:DDA": shimmer.dda,
        "NHR": 1 / hnr.mean if hnr.mean != 0 else 0,  # Approximation
        "HNR": hnr.mean,
    }
    return features

features = extract_voice_features("Recording.wav")
print(features)

Object type: Sound
Object name: <no name>
Date: Tue Apr 29 13:00:47 2025

Number of channels: 2 (stereo)
Time domain:
   Start time: 0 seconds
   End time: 3.2213333333333334 seconds
   Total duration: 3.2213333333333334 seconds
Time sampling:
   Number of samples: 154624
   Sampling period: 2.0833333333333333e-05 seconds
   Sampling frequency: 48000 Hz
   First sample centred at: 1.0416666666666666e-05 seconds
Amplitude:
   Minimum: -0.043762207 Pascal
   Maximum: 0.085357666 Pascal
   Mean: 7.84531334e-07 Pascal
   Root-mean-square: 0.00745222958 Pascal
Total energy: 0.000178899085 Pascal² sec (energy in air: 4.47247711e-07 Joule/m²)
Mean power (intensity) in air: 1.38839314e-07 Watt/m² = 51.43 dB
Standard deviation in channel 1: 0.00745225364 Pascal
Standard deviation in channel 2: 0.00745225364 Pascal



AttributeError: 'parselmouth.Sound' object has no attribute 'to_jitter'

In [93]:
import parselmouth

def extract_voice_features(file_path):
    # Load the sound file
    sound = parselmouth.Sound(file_path)
    
    # Extract pitch and pulses (PointProcess)
    pitch = sound.to_pitch()
    pulses = parselmouth.praat.call(sound, "To PointProcess (periodic, cc)", pitch, 75, 600)
    
    # Extract jitter measures (now using PointProcess)
    jitter = parselmouth.praat.call(pulses, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    jitter_abs = parselmouth.praat.call(pulses, "Get jitter (absolute)", 0, 0, 0.0001, 0.02, 1.3)
    jitter_rap = parselmouth.praat.call(pulses, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    jitter_ppq5 = parselmouth.praat.call(pulses, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    jitter_ddp = parselmouth.praat.call(pulses, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    
    # Extract shimmer measures (requires Sound + PointProcess)
    shimmer = parselmouth.praat.call([sound, pulses], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_db = parselmouth.praat.call([sound, pulses], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_apq3 = parselmouth.praat.call([sound, pulses], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_apq5 = parselmouth.praat.call([sound, pulses], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_apq11 = parselmouth.praat.call([sound, pulses], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer_dda = parselmouth.praat.call([sound, pulses], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    
    # Extract harmonicity (HNR)
    harmonicity = sound.to_harmonicity()
    hnr = harmonicity.values[harmonicity.values != -200].mean()  # Exclude Praat's NaN (-200)
    nhr = 1 / hnr if hnr != 0 else 0  # Noise-to-Harmonic Ratio (approximation)
    
    features = {
        "Jitter(%)": jitter,
        "Jitter(Abs)": jitter_abs,
        "Jitter:RAP": jitter_rap,
        "Jitter:PPQ5": jitter_ppq5,
        "Jitter:DDP": jitter_ddp,
        "Shimmer(%)": shimmer,
        "Shimmer(dB)": shimmer_db,
        "Shimmer:APQ3": shimmer_apq3,
        "Shimmer:APQ5": shimmer_apq5,
        "Shimmer:APQ11": shimmer_apq11,
        "Shimmer:DDA": shimmer_dda,
        "NHR": nhr,
        "HNR": hnr,
        "RPDE": "Requires custom computation (e.g., pyAudioAnalysis)",
        "DFA": "Requires custom computation (e.g., pyAudioAnalysis)",
        "PPE": "Requires custom computation (e.g., pyAudioAnalysis)",
    }
    return features

# Example usage
features = extract_voice_features("Recording.wav")
for key, value in features.items():
    print(f"{key}: {value}")

ValueError: Cannot convert argument "<parselmouth.Pitch object at 0x00000211C913C970>" to a known Praat argument type